In [5]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoModelForImageClassification, AutoFeatureExtractor
from PIL import Image
import nltk
from nltk.translate.bleu_score import sentence_bleu


In [ ]:
# Image folder path
folder_path = 'enter your folder path here'

In [ ]:
# Creating Dataset class
class ImageDataset(Dataset):
    def __init__(self, folder_path, transform):
        self.folder_path = folder_path
        self.transform = transform
        self.image_list = [
            os.path.join(folder_path, filename)
            for filename in os.listdir(folder_path)
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))
        ]
        print(f"Imported {len(self.image_list)} images in {folder_path}")

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_path = self.image_list[idx]
        image = Image.open(image_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Transformations for the dataset
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageDataset(folder_path, transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loading feature extractor and model
feature_extractor = AutoFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = AutoModelForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=8, ignore_mismatched_sizes=True)

# Training setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

for epoch in range(5):
    model.train()
    total_loss = 0
    predictions = []
    labels = []

    for batch in data_loader:
        images = batch.to(device)
        optimizer.zero_grad()
        outputs = model(images)

        # Generating random labels for the dummy dataset
        target_labels = torch.randint(0, 8, (images.size(0),), device=device)
        loss = criterion(outputs.logits, target_labels)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Collecting predictions and actual labels
        predictions.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy().tolist())
        labels.extend(target_labels.cpu().numpy().tolist())

    print(f'Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}')

    # BLEU Score Calculation
    references = [[str(label)] for label in labels]
    candidates = [str(prediction) for prediction in predictions]

    bleu_score = sentence_bleu(references, candidates)
    print(f'Epoch {epoch+1}, BLEU score: {bleu_score}')